In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.layers import *
from keras.models import *
from keras.utils import *
from sklearn.preprocessing import *
import keras.backend as K

Using TensorFlow backend.
C:\Users\admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
China_concat = pd.read_csv('../data/Concat/China_concat.csv', header=0)
Asia_concat = pd.read_csv('../data/Concat/Asia_concat.csv', header=0)
America_concat = pd.read_csv('../data/Concat/America_concat.csv', header=0)
Africa_concat = pd.read_csv('../data/Concat/Africa_concat.csv', header=0)
Europe_concat = pd.read_csv('../data/Concat/Europe_concat.csv', header=0)
Oceania_concat = pd.read_csv('../data/Concat/Oceania_concat.csv', header=0)

In [3]:
Test = pd.read_csv('../data/Test/Africa_test_data.csv')
Test['Date'] = [20200429, 20200430, 20200501,20200502,20200503,20200504,20200505]
Test.set_index('Date',inplace=True)

In [4]:
df = pd.DataFrame()

Africa_concat.set_index('Date',inplace=True)
label = Africa_concat[Africa_concat.columns[-1]]
#test_df = label[-7:]
Africa_concat = Africa_concat.loc[20200101:20200428]


df['Confirm'] = (Africa_concat['Daily_confirmed'] + Africa_concat['Daily_deaths']) / Africa_concat['Total_confirmed']
df['Flight'] = Africa_concat['Passenger_arrivals'] / Africa_concat['Flight_arrivals']
df['Ship'] = ( Africa_concat['Ship_crew(korea)'] + Africa_concat['Ship_crew(foreign)'] + Africa_concat['Ship_passenger'] ) / Africa_concat['Ship_count']
df['News'] = Africa_concat['News_Negative'] / Africa_concat['News_Sum']
#df['Date'] = Asia_concat['Date'].astype(int)
    
df.fillna(0, inplace = True)
#df.set_index('Date',inplace=True)



In [5]:
sc = MinMaxScaler()

df['Confirm'] = sc.fit_transform(pd.DataFrame(df['Confirm']))
df['Flight'] = sc.fit_transform(pd.DataFrame(df['Flight']))
df['Ship'] = sc.fit_transform(pd.DataFrame(df['Ship']))
df['News'] = sc.fit_transform(pd.DataFrame(df['News']))

In [6]:
df = pd.concat([df,Test])

In [7]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
        
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out+1):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
        
    return agg

In [8]:
df_s = series_to_supervised(df, 14, 0)

In [9]:
#df_s = df

In [10]:
X_train = np.array(df_s.loc[:20200428])
X_test = np.array(df_s.loc[20200429:])

y_train = np.array(label.loc[20200115:20200428])
y_test = np.array(label.loc[20200429:20200505])

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

In [11]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape,

((105, 60, 1), (7, 60, 1), (105,), (7,))

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, LSTM, Bidirectional
from keras.callbacks import EarlyStopping
from keras.models import load_model


K.clear_session()     # 모델 생성전에 tensorflow의 graph 영역을 clear한다.
xInput = Input(batch_shape=(None, X_train.shape[1], X_train.shape[2]))
xLstm_1 = LSTM(20, return_sequences = True, activation='tanh', recurrent_activation='sigmoid')(xInput)
#xLstm_2 = LSTM(20, return_sequences = True, activation='tanh', recurrent_activation='sigmoid')(xLstm_1)
#xLstm_3 = LSTM(50, return_sequences = True, activation='tanh', recurrent_activation='sigmoid')(xLstm_2)
xLstm_4 = Bidirectional(LSTM(20))(xLstm_1)
xOutput = Dense(1)(xLstm_4)


model = Model(xInput, xOutput)
model.compile(loss='mse', optimizer='adam', metrics = [ 'mae' ])


 # 학습
early_stop = EarlyStopping(monitor='loss', patience=10, mode='min', verbose=2)

model.fit(X_train, y_train, epochs=2000, batch_size=1, verbose=2, shuffle=False)



 # 예측
y_hat = model.predict(X_test, batch_size=1)


Epoch 1/2000
 - 3s - loss: 0.0480 - mae: 0.0600
Epoch 2/2000
 - 2s - loss: 0.0478 - mae: 0.0568
Epoch 3/2000
 - 2s - loss: 0.0477 - mae: 0.0559
Epoch 4/2000
 - 2s - loss: 0.0475 - mae: 0.0551
Epoch 5/2000
 - 2s - loss: 0.0473 - mae: 0.0548
Epoch 6/2000
 - 2s - loss: 0.0472 - mae: 0.0549
Epoch 7/2000
 - 2s - loss: 0.0469 - mae: 0.0548
Epoch 8/2000
 - 3s - loss: 0.0466 - mae: 0.0549
Epoch 9/2000
 - 3s - loss: 0.0463 - mae: 0.0548
Epoch 10/2000
 - 2s - loss: 0.0457 - mae: 0.0557
Epoch 11/2000
 - 2s - loss: 0.0452 - mae: 0.0574
Epoch 12/2000
 - 2s - loss: 0.0448 - mae: 0.0609
Epoch 13/2000
 - 3s - loss: 0.0445 - mae: 0.0611
Epoch 14/2000
 - 2s - loss: 0.0436 - mae: 0.0644
Epoch 15/2000
 - 2s - loss: 0.0431 - mae: 0.0653
Epoch 16/2000
 - 2s - loss: 0.0423 - mae: 0.0700
Epoch 17/2000
 - 2s - loss: 0.0412 - mae: 0.0727
Epoch 18/2000
 - 2s - loss: 0.0402 - mae: 0.0757
Epoch 19/2000
 - 2s - loss: 0.0389 - mae: 0.0769
Epoch 20/2000
 - 2s - loss: 0.0379 - mae: 0.0830
Epoch 21/2000
 - 2s - loss: 

 - 2s - loss: 0.0011 - mae: 0.0153
Epoch 161/2000
 - 2s - loss: 7.1778e-04 - mae: 0.0128
Epoch 162/2000
 - 2s - loss: 3.5131e-04 - mae: 0.0121
Epoch 163/2000
 - 2s - loss: 2.7912e-04 - mae: 0.0106
Epoch 164/2000
 - 2s - loss: 4.0918e-04 - mae: 0.0124
Epoch 165/2000
 - 2s - loss: 8.4576e-04 - mae: 0.0139
Epoch 166/2000
 - 2s - loss: 0.0019 - mae: 0.0201
Epoch 167/2000
 - 2s - loss: 0.0089 - mae: 0.0405
Epoch 168/2000
 - 2s - loss: 0.0247 - mae: 0.0681
Epoch 169/2000
 - 2s - loss: 0.0288 - mae: 0.0602
Epoch 170/2000
 - 2s - loss: 0.0056 - mae: 0.0394
Epoch 171/2000
 - 2s - loss: 0.0020 - mae: 0.0260
Epoch 172/2000
 - 2s - loss: 9.8290e-04 - mae: 0.0215
Epoch 173/2000
 - 2s - loss: 6.9964e-04 - mae: 0.0180
Epoch 174/2000
 - 2s - loss: 4.6846e-04 - mae: 0.0162
Epoch 175/2000
 - 2s - loss: 4.1461e-04 - mae: 0.0150
Epoch 176/2000
 - 2s - loss: 3.2929e-04 - mae: 0.0138
Epoch 177/2000
 - 2s - loss: 3.2473e-04 - mae: 0.0134
Epoch 178/2000
 - 2s - loss: 2.8269e-04 - mae: 0.0126
Epoch 179/2000
 -

Epoch 316/2000
 - 2s - loss: 9.5482e-05 - mae: 0.0076
Epoch 317/2000
 - 2s - loss: 9.6919e-05 - mae: 0.0076
Epoch 318/2000
 - 2s - loss: 9.7990e-05 - mae: 0.0075
Epoch 319/2000
 - 3s - loss: 1.3516e-04 - mae: 0.0082
Epoch 320/2000
 - 2s - loss: 1.9261e-04 - mae: 0.0086
Epoch 321/2000
 - 2s - loss: 3.3208e-04 - mae: 0.0101
Epoch 322/2000
 - 3s - loss: 2.8961e-04 - mae: 0.0100
Epoch 323/2000
 - 2s - loss: 5.1897e-04 - mae: 0.0118
Epoch 324/2000
 - 2s - loss: 7.7007e-04 - mae: 0.0142
Epoch 325/2000
 - 2s - loss: 7.1531e-04 - mae: 0.0138
Epoch 326/2000
 - 2s - loss: 4.7372e-04 - mae: 0.0127
Epoch 327/2000
 - 2s - loss: 2.1374e-04 - mae: 0.0095
Epoch 328/2000
 - 2s - loss: 1.3632e-04 - mae: 0.0079
Epoch 329/2000
 - 2s - loss: 9.9148e-05 - mae: 0.0070
Epoch 330/2000
 - 2s - loss: 8.4273e-05 - mae: 0.0068
Epoch 331/2000
 - 2s - loss: 7.1957e-05 - mae: 0.0063
Epoch 332/2000
 - 3s - loss: 7.0560e-05 - mae: 0.0062
Epoch 333/2000
 - 3s - loss: 6.5885e-05 - mae: 0.0060
Epoch 334/2000
 - 2s - loss:

Epoch 469/2000
 - 3s - loss: 9.4394e-04 - mae: 0.0083
Epoch 470/2000
 - 3s - loss: 5.6667e-04 - mae: 0.0097
Epoch 471/2000
 - 2s - loss: 3.4225e-04 - mae: 0.0092
Epoch 472/2000
 - 3s - loss: 4.9774e-04 - mae: 0.0098
Epoch 473/2000
 - 2s - loss: 0.0055 - mae: 0.0300
Epoch 474/2000
 - 3s - loss: 0.0023 - mae: 0.0232
Epoch 475/2000
 - 3s - loss: 2.1171e-04 - mae: 0.0106
Epoch 476/2000
 - 3s - loss: 1.1072e-04 - mae: 0.0076
Epoch 477/2000
 - 3s - loss: 7.1769e-05 - mae: 0.0063
Epoch 478/2000
 - 2s - loss: 4.9147e-05 - mae: 0.0053
Epoch 479/2000
 - 3s - loss: 4.1981e-05 - mae: 0.0049
Epoch 480/2000
 - 3s - loss: 3.5244e-05 - mae: 0.0046
Epoch 481/2000
 - 3s - loss: 3.2915e-05 - mae: 0.0044
Epoch 482/2000
 - 3s - loss: 2.9454e-05 - mae: 0.0042
Epoch 483/2000
 - 3s - loss: 2.9046e-05 - mae: 0.0042
Epoch 484/2000
 - 3s - loss: 2.7065e-05 - mae: 0.0040
Epoch 485/2000
 - 3s - loss: 2.9865e-05 - mae: 0.0041
Epoch 486/2000
 - 3s - loss: 3.2259e-05 - mae: 0.0041
Epoch 487/2000
 - 3s - loss: 6.1874e

Epoch 624/2000
 - 2s - loss: 4.5778e-05 - mae: 0.0053
Epoch 625/2000
 - 3s - loss: 4.1567e-05 - mae: 0.0049
Epoch 626/2000
 - 3s - loss: 3.6625e-05 - mae: 0.0048
Epoch 627/2000
 - 3s - loss: 3.5893e-05 - mae: 0.0047
Epoch 628/2000
 - 3s - loss: 3.3564e-05 - mae: 0.0046
Epoch 629/2000
 - 3s - loss: 3.5371e-05 - mae: 0.0046
Epoch 630/2000
 - 3s - loss: 3.7202e-05 - mae: 0.0046
Epoch 631/2000
 - 3s - loss: 5.2017e-05 - mae: 0.0050
Epoch 632/2000
 - 3s - loss: 8.7446e-05 - mae: 0.0055
Epoch 633/2000
 - 3s - loss: 2.1908e-04 - mae: 0.0074
Epoch 634/2000
 - 3s - loss: 4.7245e-04 - mae: 0.0102
Epoch 635/2000
 - 3s - loss: 9.9088e-04 - mae: 0.0126
Epoch 636/2000
 - 3s - loss: 9.2041e-04 - mae: 0.0164
Epoch 637/2000
 - 3s - loss: 0.0014 - mae: 0.0185
Epoch 638/2000
 - 3s - loss: 6.3723e-04 - mae: 0.0166
Epoch 639/2000
 - 3s - loss: 5.4854e-04 - mae: 0.0106
Epoch 640/2000
 - 3s - loss: 3.4533e-04 - mae: 0.0106
Epoch 641/2000
 - 3s - loss: 1.7952e-04 - mae: 0.0085
Epoch 642/2000
 - 3s - loss: 6.7

In [ ]:
a_axis = np.arange(0, len(y_train))
b_axis = np.arange(len(y_train), len(y_train) + len(y_hat))

plt.figure(figsize=(10,6))
plt.plot(a_axis, y_train.reshape(105,), 'o-')
plt.plot(b_axis, y_hat.reshape(7,), 'o-', color='red', label='Predicted')
plt.plot(b_axis, y_test.reshape(7,), 'o-', color='green', alpha=0.2, label='Actual')
plt.legend()
plt.show()

In [ ]:
y_test, y_hat

In [ ]:
y_hat

In [ ]:
y_test.sum(), y_hat.sum()

In [ ]:
model.save('../model/Africa_model.h5')